In [1]:
import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt 
import matplotlib.font_manager as fm  

!apt-get update -qq
!apt-get install fonts-nanum* -qq

font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()
matplotlib.rcParams['axes.unicode_minus'] = False

!pip install catboost
!pip install pycaret
from tqdm.notebook import tqdm


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

NanumBarunGothic
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
!pip install markupsafe==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.0.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (31 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import jinja2
from pycaret.regression import *

In [4]:
import pandas as pd

train = pd.read_csv('/content/eng_train.csv')
test = pd.read_csv('/content/eng_test.csv')

submission = pd.read_csv('/content/eng_sample.csv')

In [5]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [6]:
reduce_mem_usage(train)
reduce_mem_usage(test)
reduce_mem_usage(submission)

Memory usage of dataframe is 1.69 MB
Memory usage after optimization is: 1.09 MB
Decreased by 35.7%
Memory usage of dataframe is 0.36 MB
Memory usage after optimization is: 0.26 MB
Decreased by 29.2%
Memory usage of dataframe is 0.12 MB
Memory usage after optimization is: 0.02 MB
Decreased by 81.2%


,index,counts
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
7915,7915,0
7916,7916,0
7917,7917,0
7918,7918,0


## Feature Engineering

In [7]:
#one-hot encoding

train_one = pd.get_dummies(train)
test_one =  pd.get_dummies(test)

In [8]:
train_one['Send_grid'] = np.log1p(train_one['Send_grid']) ##데이터 셋이 살짝 달라져서 로그가 아닌 정규화를 진행해주시거나 해당 부분을 날리는 게 더 좋아보였습니다. 
test_one['Send_grid'] = np.log1p(test_one['Send_grid'])  ##이전 데이터 셋도 생각해보니 해당 특성을 버릴 생각은 안 했었네요..? 
                                                                                    ##많은 방법이 있을 것 같긴 합니다. 다양한 실험을 추천드립니다. 
train_one['Rec_grid'] = np.log1p(train_one['Rec_grid'])
test_one['Rec_grid'] = np.log1p(test_one['Rec_grid'])


In [9]:
train_one

,index,Send,Rec,counts,Send_grid,Rec_grid,item_DIY자재/용품,item_PC,item_PC주변기기,item_가공식품,item_가방,item_건강식품,item_건강용품,item_게임기/타이틀,item_계절가전,item_골프,item_공구,item_과자,item_구강위생용품,item_기능성,item_기저귀/물티슈,item_기타디지털/가전,item_기타스포츠/레저,item_기타식품,item_기타출산/육아,item_기타패션의류,item_기타패션잡화,item_기타화장품/미용,item_김치,item_낚시,item_남성화장품,item_냉동/간편조리식품,item_네일케어,item_농산물,item_눈관리용품,item_다이어트식품,item_등산,item_모니터,item_모자,item_문구/사무용품,item_문화컨텐츠,item_바디케어,item_반려동물,item_반찬,item_베이스메이크업,item_분유/이유식/아기간식,item_뷰티소품,item_상의,item_색조메이크업,item_생활가전,item_생활용품,item_서재/사무용가구,item_선글라스/안경테,item_선케어,item_세탁용품,item_수납/정리용품,item_수납가구,item_수산,item_스마트디바이스,item_스마트디바이스액세서리,item_스킨케어,item_스포츠잡화,item_신발,item_아우터,item_아웃도어가구,item_양말/스타킹,item_언더웨어,item_완구/매트,item_욕실용품,item_원피스/점프슈트,item_위생/건강용품,item_유아가구,item_음료,item_음반,item_음향가전,item_의료용품,item_이미용가전,item_인테리어소품,item_자동차용품,item_잠옷,item_재활운동용품,item_주방가구,item_주방가전,item_주방용품,item_주얼리,item_지갑,item_축산,item_출산/유아동의류,item_출산/유아동잡화,item_취미용품,item_침구단품,item_침구세트,item_침실가구,item_카페트/러그,item_캠핑,item_커튼/블라인드,item_클렌징,item_태블릿PC/노트북액세서리,item_패션소품,item_하의,item_향수,item_헤어스타일링,item_헤어액세서리,item_헤어케어,item_헬스,item_홈데코
0,0,5011000595017300,2871000192069300,3,10.821996,10.265036,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,4148000690043300,5011000264024400,3,10.632991,10.821996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,5011000078068400,1120000007005400,3,10.821996,9.323758,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,4127100048006400,5011000587019400,7,10.627940,10.821996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,5011000078068400,2823700010076300,3,10.821996,10.248424,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31679,31679,4471000290087200,5011000213073200,3,10.707975,10.821996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31680,31680,1129000014045300,5011000319087100,4,9.331761,10.821996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31681,31681,1129000014045300,5011000263065200,6,9.331761,10.821996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31682,31682,4127300065073100,5011000264061200,7,10.627988,10.821996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
test_one

,index,Send,Rec,Send_grid,Rec_grid,item_DIY자재/용품,item_PC,item_PC주변기기,item_가공식품,item_가방,item_건강식품,item_건강용품,item_게임기/타이틀,item_계절가전,item_골프,item_공구,item_과자,item_구강위생용품,item_기능성,item_기저귀/물티슈,item_기타디지털/가전,item_기타스포츠/레저,item_기타식품,item_기타출산/육아,item_기타패션의류,item_기타패션잡화,item_기타화장품/미용,item_김치,item_낚시,item_남성화장품,item_냉동/간편조리식품,item_네일케어,item_농산물,item_눈관리용품,item_다이어트식품,item_등산,item_모니터,item_모자,item_문구/사무용품,item_문화컨텐츠,item_바디케어,item_반려동물,item_반찬,item_베이스메이크업,item_분유/이유식/아기간식,item_뷰티소품,item_상의,item_색조메이크업,item_생활가전,item_생활용품,item_서재/사무용가구,item_선글라스/안경테,item_선케어,item_세탁용품,item_수납/정리용품,item_수납가구,item_수산,item_스마트디바이스,item_스마트디바이스액세서리,item_스킨케어,item_스포츠잡화,item_신발,item_아우터,item_아웃도어가구,item_양말/스타킹,item_언더웨어,item_완구/매트,item_욕실용품,item_원피스/점프슈트,item_위생/건강용품,item_유아가구,item_음료,item_음반,item_음향가전,item_의료용품,item_이미용가전,item_인테리어소품,item_자동차용품,item_잠옷,item_재활운동용품,item_주방가구,item_주방가전,item_주방용품,item_주얼리,item_지갑,item_축산,item_출산/유아동의류,item_출산/유아동잡화,item_취미용품,item_침구단품,item_침구세트,item_침실가구,item_카페트/러그,item_캠핑,item_커튼/블라인드,item_클렌징,item_태블릿PC/노트북액세서리,item_패션소품,item_하의,item_향수,item_헤어스타일링,item_헤어액세서리,item_헤어케어,item_헬스,item_홈데코
0,0,4167000577042200,5011000435014100,10.637561,10.821996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1156000009012200,5011000172034400,9.355393,10.821996,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,4122000363057300,5011000361097300,10.626703,10.821996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,3,5011000436041400,2826000084036400,10.821996,10.249238,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,4150000241065200,5011000169044300,10.633473,10.821996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,7915,5011000266051200,4623000417038100,10.821996,10.741406,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7916,7916,1154500001098300,5011000264055100,9.354094,10.821996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7917,7917,5013000610049100,1147000018091400,10.822395,9.347577,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7918,7918,5013000610049100,3117000039026100,10.822395,10.347243,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
train_one.corr()['counts'].sort_values()  


item_농산물             -0.110292
Send                 -0.045961
Send_grid            -0.035819
item_상의              -0.016785
item_하의              -0.016054
item_음반              -0.015856
item_아우터             -0.014013
item_반려동물            -0.007015
item_원피스/점프슈트        -0.006109
item_커튼/블라인드         -0.005259
item_수납가구            -0.005070
item_이미용가전           -0.004293
item_색조메이크업          -0.004281
item_인테리어소품          -0.004151
item_주방가구            -0.004058
item_침구단품            -0.004043
item_출산/유아동의류        -0.003857
item_가방              -0.003832
item_DIY자재/용품        -0.003557
item_모자              -0.003538
item_눈관리용품           -0.003506
item_모니터             -0.003303
item_세탁용품            -0.003149
item_분유/이유식/아기간식     -0.003076
item_네일케어            -0.002814
item_언더웨어            -0.002807
item_등산              -0.002713
item_스포츠잡화           -0.002710
item_유아가구            -0.002710
item_PC주변기기          -0.002656
item_양말/스타킹          -0.002341
item_잠옷              -0.002341
item_패션소

### Pycaret

In [12]:
train_one.columns

Index(['index', 'Send', 'Rec', 'counts', 'Send_grid', 'Rec_grid',
       'item_DIY자재/용품', 'item_PC', 'item_PC주변기기', 'item_가공식품',
       ...
       'item_클렌징', 'item_태블릿PC/노트북액세서리', 'item_패션소품', 'item_하의', 'item_향수',
       'item_헤어스타일링', 'item_헤어액세서리', 'item_헤어케어', 'item_헬스', 'item_홈데코'],
      dtype='object', length=106)

In [13]:
train_one.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31684 entries, 0 to 31683
Columns: 106 entries, index to item_홈데코
dtypes: float64(2), int16(2), int64(2), uint8(100)
memory usage: 4.1 MB


In [14]:
train_one.dtypes

index                   int16
Send                    int64
Rec                     int64
counts                  int16
Send_grid             float64
Rec_grid              float64
item_DIY자재/용품           uint8
item_PC                 uint8
item_PC주변기기             uint8
item_가공식품               uint8
item_가방                 uint8
item_건강식품               uint8
item_건강용품               uint8
item_게임기/타이틀            uint8
item_계절가전               uint8
item_골프                 uint8
item_공구                 uint8
item_과자                 uint8
item_구강위생용품             uint8
item_기능성                uint8
item_기저귀/물티슈            uint8
item_기타디지털/가전           uint8
item_기타스포츠/레저           uint8
item_기타식품               uint8
item_기타출산/육아            uint8
item_기타패션의류             uint8
item_기타패션잡화             uint8
item_기타화장품/미용           uint8
item_김치                 uint8
item_낚시                 uint8
item_남성화장품              uint8
item_냉동/간편조리식품          uint8
item_네일케어               uint8
item_농산물  

In [34]:
train_ones = train_one.astype('float')
test_ones = test_one.astype('float')

In [31]:
reduce_mem_usage(train_ones)
reduce_mem_usage(test_ones)


Memory usage of dataframe is 12.81 MB
Memory usage after optimization is: 3.44 MB
Decreased by 73.1%
Memory usage of dataframe is 3.17 MB
Memory usage after optimization is: 0.85 MB
Decreased by 73.3%


,index,Send,Rec,Send_grid,Rec_grid,item_DIY자재/용품,item_PC,item_PC주변기기,item_가공식품,item_가방,...,item_클렌징,item_태블릿PC/노트북액세서리,item_패션소품,item_하의,item_향수,item_헤어스타일링,item_헤어액세서리,item_헤어케어,item_헬스,item_홈데코
0,0,1831626520,1961228756,10,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,971359208,1698249056,9,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,-9998188,1887311956,10,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1962256056,1797581616,10,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,976207088,1695258956,10,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,7915,1792265856,698838804,10,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7916,7916,-92968388,1790269756,9,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7917,7917,681503820,-1063076472,10,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7918,7918,681503820,538398132,10,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
train_ones.dtypes


index          float64
Send           float64
Rec            float64
counts         float64
Send_grid      float64
                ...   
item_헤어스타일링    float64
item_헤어액세서리    float64
item_헤어케어      float64
item_헬스        float64
item_홈데코       float64
Length: 106, dtype: object

### GBR


In [47]:

from sklearn.ensemble import GradientBoostingRegressor

train_X = train_ones.drop(['index', 'counts'], axis=1)
train_Y = train_ones['counts']

gbr = GradientBoostingRegressor()
gbr.fit(train_X,train_Y)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [48]:
test_ones = test_ones.drop('index', axis=1)

In [49]:
pred =  gbr.predict(test_ones)

In [50]:
submission['운송장_건수'] = pred

In [51]:
submission

,index,운송장_건수
0,0,14.221520
1,1,6.448792
2,2,5.113077
3,3,5.548155
4,4,5.517250
...,...,...
7915,7915,4.393714
7916,7916,5.323135
7917,7917,4.240864
7918,7918,4.311670


In [52]:
submission.to_csv('./gbr.csv',index = False)